In [43]:
import time
import requests
import pandas as pd
from datetime import date, timedelta

## 테스트 1

In [34]:
access_token = 'EAAOcZBdUexHkBOZBguMYcuyKgocALJjwDtnghyyZAZAu3ZAdFXmDs5mWbGJmKtu6no3N2tkPZCLg9bjx9Ld9SLR1bgPIPSZCQqE5p7jzM1RJhalpo76yUzTZBlJBOBZAXoNdPmyuZAk3eRHBYFkGsfzBLtZCqDhxS3BrjRokWm9Wo3jh7S6PK1TLeqgqyUoRBMkkpH9'
ad_account_id = 'act_410814326151786'
api_version = 'v20.0'
fields = 'campaign_name,adset_name,ad_name,impressions,clicks,spend'

In [35]:
# API 요청 URL
url = f'https://graph.facebook.com/{api_version}/{ad_account_id}/insights?use_unified_attribution_setting=true&breakdowns=publisher_platform&fields={fields}&access_token={access_token}'

In [36]:
# API 호출
response = requests.get(url)

In [37]:
# 응답 처리
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f'Error: {response.status_code}')
    print(response.text)

{'data': [{'impressions': '156', 'clicks': '0', 'spend': '449', 'date_start': '2024-05-11', 'date_stop': '2024-06-09', 'publisher_platform': 'audience_network'}, {'impressions': '15658', 'clicks': '67', 'spend': '25257', 'date_start': '2024-05-11', 'date_stop': '2024-06-09', 'publisher_platform': 'facebook'}, {'impressions': '598795', 'clicks': '19600', 'spend': '2309441', 'date_start': '2024-05-11', 'date_stop': '2024-06-09', 'publisher_platform': 'instagram'}, {'impressions': '5', 'clicks': '0', 'spend': '0', 'date_start': '2024-05-11', 'date_stop': '2024-06-09', 'publisher_platform': 'messenger'}], 'paging': {'cursors': {'before': 'MAZDZD', 'after': 'MwZDZD'}}}


In [38]:
# API 응답 데이터
data = response.json()

# 데이터프레임으로 변환
df = pd.DataFrame(data['data'])

# 데이터프레임 출력
df

,impressions,clicks,spend,date_start,date_stop,publisher_platform
0,156,0,449,2024-05-11,2024-06-09,audience_network
1,15658,67,25257,2024-05-11,2024-06-09,facebook
2,598795,19600,2309441,2024-05-11,2024-06-09,instagram
3,5,0,0,2024-05-11,2024-06-09,messenger


---------

## 테스트 2

In [60]:
ACCESS_TOKEN = 'EAAOcZBdUexHkBOZBguMYcuyKgocALJjwDtnghyyZAZAu3ZAdFXmDs5mWbGJmKtu6no3N2tkPZCLg9bjx9Ld9SLR1bgPIPSZCQqE5p7jzM1RJhalpo76yUzTZBlJBOBZAXoNdPmyuZAk3eRHBYFkGsfzBLtZCqDhxS3BrjRokWm9Wo3jh7S6PK1TLeqgqyUoRBMkkpH9'
ACT_ID = 'act_410814326151786'
APP_ID = '1017021766485113'
API_VERSION = 'v20.0'
FIELDS = 'date_start,date_stop,ad_name,campaign_name,adset_name,impressions,spend,inline_link_clicks,actions,action_values'

In [61]:
#조회할 데이터의 기간 정보
today = date.today()
start_date = str(today - timedelta(days=31))
end_date = str(today - timedelta(days=1))

In [64]:
def post_meta_ads_data(access_token, api_version, act_id, start_date, end_date):
    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)

    date_of_url = '['
    while(start_date<=end_date):
        date_of_url = date_of_url + '{"since":"'+str(start_date)+'","until":"'+str(start_date)+'"},'
        start_date = start_date + timedelta(days=1)
    date_of_url = date_of_url + ']'

    url = "https://graph.facebook.com/" + api_version + "/" + act_id + "/insights?use_unified_attribution_setting=true&breakdowns=publisher_platform&fields=" + FIELDS \
          + "&level=ad" \
          + '&time_ranges=' + date_of_url \
          + '&access_token=' + access_token

    report = requests.post(url = url)

    return report.json()["report_run_id"]
    
def check_meta_data_report(report_id,access_token,api_version):
    check_report = requests.get('https://graph.facebook.com/' + api_version + '/' + report_id + '?access_token=' + access_token)
    
    return check_report.json()["async_status"]

def get_meta_data_to_df(report_id,access_token,api_version):
    meta_date = [] #날짜
    meta_campaign = [] #캠페인명
    meta_adset = [] #애드세트명
    meta_ad = [] #애드명
    meta_publisher_platform = [] #플랫폼
    meta_imp = [] #노출
    meta_spend = [] #소진비용
    meta_clicks = [] #클릭
    meta_install = [] #인스톨
    meta_purchase = [] #구매
    meta_purchase_value = [] #구매값

    response = requests.get('https://graph.facebook.com/' + api_version + '/' + report_id + '/insights?access_token=' + access_token)
    meta_data = response.json()

    while (1):
        for i in range(len(meta_data["data"])):
            meta_date.append(meta_data["data"][i]["date_start"])
            meta_campaign.append(meta_data["data"][i]["campaign_name"])
            meta_adset.append(meta_data["data"][i]["adset_name"])
            meta_ad.append(meta_data["data"][i]["ad_name"])
            meta_publisher_platform.append(meta_data["data"][i]["publisher_platform"])

            if 'impressions' in meta_data["data"][i].keys():
                meta_imp.append(meta_data["data"][i]["impressions"])
            else:
                meta_imp.append(0)

            if 'spend' in meta_data["data"][i].keys():
                meta_spend.append(meta_data["data"][i]["spend"])
            else:
                meta_spend.append(0)

            if 'inline_link_clicks' in meta_data["data"][i].keys():
                meta_clicks.append(meta_data["data"][i]["inline_link_clicks"])
            else:
                meta_clicks.append(0)

            if 'actions' in meta_data["data"][i].keys():
                meta_install_in, meta_purchase_in = 0, 0
                for j in range(len(meta_data["data"][i]["actions"])):
                    if meta_data["data"][i]["actions"][j]["action_type"] == "omni_app_install":
                        meta_install_in = meta_data["data"][i]["actions"][j]["value"]
                    elif meta_data["data"][i]["actions"][j]["action_type"] == "omni_purchase":
                        meta_purchase_in = meta_data["data"][i]["actions"][j]["value"]

                meta_install.append(meta_install_in)
                meta_purchase.append(meta_purchase_in)
            else:
                meta_install.append(0)
                meta_purchase.append(0)

            if 'action_values' in meta_data["data"][i].keys():
                meta_purchase_value_in = 0
                for j in range(len(meta_data["data"][i]["action_values"])):
                    if meta_data["data"][i]["action_values"][j]["action_type"] == "omni_purchase":
                        meta_purchase_value_in = meta_data["data"][i]["action_values"][j]["value"]

                meta_purchase_value.append(meta_purchase_value_in)
            else:
                meta_purchase_value.append(0)

        #netx page
        if "next" in meta_data["paging"].keys():
            response = requests.get(meta_data["paging"]["next"])
            meta_data = response.json()
        else:
            break

    col = ["Reporting starts", "Campaign name", "Ad Set Name", "Ad name", "Platform", "Impressions", "Amount spent (USD)", "Link clicks", "App installs", "Purchases", "Purchases conversion value"]
    meta_data_df = pd.DataFrame(zip(meta_date, meta_campaign, meta_adset, meta_ad, meta_publisher_platform, meta_imp, meta_spend, meta_clicks, meta_install, meta_purchase, meta_purchase_value),columns = col)
    
    return meta_data_df

if __name__ == '__main__':
    status = False
    while(status!=True):
        meta_report_id = post_meta_ads_data(ACCESS_TOKEN, API_VERSION, ACT_ID, start_date, end_date)
        for i in range(3):
            time.sleep(60)
            if check_meta_data_report(meta_report_id,ACCESS_TOKEN,API_VERSION) == 'Job Completed':
                status = True
                break

    meta_report = get_meta_data_to_df(meta_report_id,ACCESS_TOKEN,API_VERSION)
    meta_report.to_csv('.\\facebook_ads_data.csv',index=False,encoding='utf-8-sig')

Current meta_date: ['2024-05-11']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...']
Current meta_date: ['2024-05-11', '2024-05-12']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...']
Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', '0509_샵사이다 팝업스토어 릴스']
Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', '0509_샵사이다 팝업스토어 릴스', '0510_샵사이다 팝업스토어']
Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', '0509_샵사이다 팝업스토어 릴스', '0

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', '0509_샵사이다 팝업스토어 릴스', '0510_샵사이다 팝업스토어', '0510_유미의세포들 팝업스토어', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: ⏰ 팝가가 미리 알려주는 “업커밍 팝업스토어”⏰\n📝 원피스...', 'Instagram 게시물: 📢 잠실에 오픈하는 짱구 팝업스토어?!\n\n짱구는 못말려...', '0513_상쾌환 팝업스토어', '0509_샵사이다 팝업스토어 릴스', '0510_샵사이다 팝업스토어', '0510_유미의세포들 팝업스토어', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: ⏰ 팝가가 미리 알려주는 “업커밍 팝업스토어”⏰\n📝 원피스...', 'Instagram 게시물: 📢 잠실에 오픈하는 짱구 팝업스토어?!\n\n짱구는 못말려...', '0513_상쾌환 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19']
Current meta_ad: ['Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', '0509_샵사이다 팝업스토어 릴스', '0510_샵사이다 팝업스토어', '0510_유미의세포들 팝업스토어', 'Instagram 게시물: 🚨저장/\xa0공유 필수!\n📢이번 주, 성수동 핫한 팝업스토어...', 'Instagram 게시물:

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

...

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 

Current meta_date: ['2024-05-11', '2024-05-12', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-13', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-14', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-15', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-16', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-17', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-18', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-19', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-20', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-21', '2024-05-22', '2024-05-22', '2024-05-22', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', '2024-05-23', 